<a href="https://colab.research.google.com/github/anaguilarar/agwise_data_sourcing/blob/main/GEESoilGrids_data_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ag-Wise Data Sourcing

## SOIL Downloader

This tutorial explains how to use this notebook to download soil data from google earth engine

In [ ]:
import os

if not os.path.exists('agwise_data_sourcing'):
  !git clone https://github.com/anaguilarar/agwise_data_sourcing.git
  os.chdir('/content/agwise_data_sourcing')
else:
  os.chdir('/content/agwise_data_sourcing')

## Workflow Overview
1. **Country Example Configuration** – Select area and soil property.
2. **Soil Data visualization** – Visualize the soil property.

### Country Example Configuration


In this section, you will set the parameters for your analysis. Modify the dictionary below to match your region and product of interest.


- `ADM0_NAME` define the administrative levels.
- `property` sets the MODIS/VIIRS dataset.


Example: soil extraction for **Kenya – Coast Province (2023)**

In [ ]:
### INITIAL configuration

configuration = {
    'GENERAL_SETTINGS':{
      'ee_project_name': 'ee-anaguilarar'
      },

    'DATA_DOWNLOAD':
     {
      'ADM0_NAME': 'Kenya',
      'ADM1_NAME': 'Kericho',
      'ADM2_NAME': None,
      'property': 'sand',

    }
}


- The first time you run this notebook, GEE will request authentication (`ee.Authenticate()`).
- Depending on your area size, the data request might take a few minutes.

In [ ]:
from gee_datasets.gee_data import GEESoilGrids
import ee
ee.Initialize(project=configuration['GENERAL_SETTINGS']['ee_project_name'])


### Data Downloading


This section connects to Google Earth Engine, defines your region of interest, and retrieves the vegetation index time series.


Steps:
1. **Initialize Google Earth Engine (GEE)** with your project.
2. **Create the downloader object** (`GEESoilGrids`).
3. **Run the query** to retrieve the imagery.

In [ ]:
data_downloader = GEESoilGrids(configuration['DATA_DOWNLOAD']['ADM0_NAME'])

data_downloader.list_of_products

{'bdod': 'projects/soilgrids-isric/bdod_mean',
 'cec': 'projects/soilgrids-isric/cec_mean',
 'cfvo': 'projects/soilgrids-isric/cfvo_mean',
 'clay': 'projects/soilgrids-isric/clay_mean',
 'sand': 'projects/soilgrids-isric/sand_mean',
 'silt': 'projects/soilgrids-isric/silt_mean',
 'nitrogen': 'projects/soilgrids-isric/nitrogen_mean',
 'soc': 'projects/soilgrids-isric/soc_mean',
 'phh2o': 'projects/soilgrids-isric/phh2o_mean',
 'wv0010': 'ISRIC/SoilGrids250m/v2_0/wv0010',
 'wv0033': 'ISRIC/SoilGrids250m/v2_0/wv0033',
 'wv1500': 'ISRIC/SoilGrids250m/v2_0/wv1500'}

In [ ]:

data_downloader.initialize_query(configuration['DATA_DOWNLOAD']['property'])


In [ ]:
band_names = data_downloader.query.bandNames().getInfo()
band_names

['sand_0-5cm_mean',
 'sand_5-15cm_mean',
 'sand_15-30cm_mean',
 'sand_30-60cm_mean',
 'sand_60-100cm_mean',
 'sand_100-200cm_mean']

### Soil Data visualization

In [ ]:
import geemap


# Create a map
Map = geemap.Map(center=[-1.37, 38.01], zoom=6)

# Define visualization parameters

vis_parameters = {'min': 50, 'max': 1000,
 'palette': ['5d5851','635a4b','6a5b44','715c3d','785e36','7e5f30','856129','8c6222','92641c','996515','a0660e','a66808','ad6901']}
# Add the image layer
Map.addLayer(data_downloader.query.select(band_names[0]), vis_parameters, band_names[0])

# Display the map
Map

Map(center=[-1.37, 38.01], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

### Download data for a specific administrative level

You can target data at different administrative levels using the configuration keys:

1. Set `ADM0_NAME` for the country (required).
2. Set `ADM1_NAME` for the first-level admin (province/state) if you want a subregion.
3. Set `ADM2_NAME` for the district/municipality if available and needed.


**Example configuration (Kenya, Coast province):**


```python
configuration['DATA_DOWNLOAD'].update({
'ADM0_NAME': 'Kenya',
'ADM1_NAME': 'Coast',
'ADM2_NAME': None,

})


In [ ]:

soil_image = data_downloader.get_adm_level_data(adm_level='ADM1', feature_name = configuration['DATA_DOWNLOAD']['ADM1_NAME'])

Map = geemap.Map(center=[-1.37, 38.01], zoom=8)

Map.addLayer(soil_image.select(band_names[0]), vis_parameters, band_names[0])

# Display the map
Map

data will be processed for: Kericho


Map(center=[-1.37, 38.01], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…